In [10]:
import cv2
import numpy as np
import winsound

#video_path = "vlc-record-2021-10-21-20h55m37s-rtsp___172.16.1.100_554_-.mp4"
#video_path = "vlc-record-2021-10-21-20h56m22s-rtsp___172.16.1.100_554_-.mp4"
video_path = "vlc-record-2021-10-21-20h57m16s-rtsp___172.16.1.100_554_-.mp4"
cap = cv2.VideoCapture(video_path)

fps = cap.get(cv2.CAP_PROP_FPS)
if fps <= 1:
    fps = 25

short_back = cv2.createBackgroundSubtractorMOG2(
    history=200,
    varThreshold=25,
    detectShadows=False
)

long_back = cv2.createBackgroundSubtractorMOG2(
    history=5000,
    varThreshold=25,
    detectShadows=False
)

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
min_area = 700

static_time = 3
static_frames_threshold = int(static_time * fps)

static_counter = {}

while True:
    ret, frame = cap.read()
    if not ret:
        break

    short_mask = short_back.apply(frame, learningRate=0.05)
    long_mask  = long_back.apply(frame,  learningRate=0.001)

    _, short_mask = cv2.threshold(short_mask, 200, 255, cv2.THRESH_BINARY)
    _, long_mask  = cv2.threshold(long_mask, 200, 255, cv2.THRESH_BINARY)

    short_mask = cv2.morphologyEx(short_mask, cv2.MORPH_OPEN, kernel, iterations=1)
    long_mask  = cv2.morphologyEx(long_mask,  cv2.MORPH_OPEN, kernel, iterations=1)

    candidate_mask = cv2.bitwise_and(long_mask, cv2.bitwise_not(short_mask))

    contours, _ = cv2.findContours(candidate_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    alarm_triggered = False

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < min_area:
            continue

        x, y, w, h = cv2.boundingRect(cnt)
        key = (x//20, y//20)

        static_counter.setdefault(key, 0)
        static_counter[key] += 1

        if static_counter[key] > static_frames_threshold:
            color = (0, 0, 255)
            alarm_triggered = True
        else:
            color = (0, 165, 255)

        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)

    if alarm_triggered:
        try:
            winsound.Beep(1000, 200)
        except:
            print("ALARM!")

    cv2.imshow("Frame", frame)
    cv2.imshow("Short (Motion)", short_mask)
    cv2.imshow("Long (New Objects)", long_mask)
    cv2.imshow("Candidate", candidate_mask)

    key = cv2.waitKey(int(1000 / fps)) & 0xFF
    if key == ord('q') or key == 27:
        break

cap.release()
cv2.destroyAllWindows()